In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time
import re

In [2]:
# Define the headers as specified
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    'Host': 'guba.eastmoney.com',
    'Sec-CH-UA': '"Not)A;Brand";v="99", "Microsoft Edge";v="127", "Chromium";v="127"',
    'Sec-CH-UA-Mobile': '?0',
    'Sec-CH-UA-Platform': '"Windows"',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36 Edg/127.0.0.0',
    'X-Forwarded-For': '218.75.123.175'
}

In [3]:
page1_baseurl = 'https://guba.eastmoney.com/remenba.aspx?type=1&tab='
page2_baseurl = 'https://guba.eastmoney.com/'
tab_list = ['1', '2', '7']


In [4]:
def get_page1_data(tab):
    page1_url = page1_baseurl + tab
    time.sleep(0.1)
    response = requests.get(page1_url, headers=headers)
    response.encoding = 'utf-8'
    html = response.text

    soup = BeautifulSoup(html, 'html.parser')

    block = soup.find('div', class_='ngbglistdiv')

    links = block.find_all('li') if block else []

    paths = []
    for li in links:
        a_tag = li.find('a')
        if a_tag and 'href' in a_tag.attrs:
            paths.append(a_tag['href'])
    return paths

    


In [5]:
path_list=get_page1_data('1')

In [6]:
class NewsInfo:
    def __init__(self, views, comments, title, author, time):
        self.title = title  # 文章标题
        self.author = author  # 作者名
        self.time = time  # 发布时间
        self.views = views  # 阅读量
        self.comments = comments  # 评论量

In [7]:
def get_date(path):
    date_url = page2_baseurl + path
    time.sleep(0.1)
    response = requests.get(date_url, headers=headers)
    response.encoding = 'utf-8'
    html = response.text

    soup = BeautifulSoup(html, 'html.parser')
    block = soup.find('div', class_='time')
    date = block.text if block else ''
    return date

def check_data(timedata):
    if timedata == '':
        return 2
    dt = datetime.strptime(timedata, '%Y-%m-%d %H:%M:%S')
    print(dt)
    if dt.year < 2022:
        return 0
    if dt.year > 2023:
        return 2
    if dt.year == 2022 or dt.year == 2023:
        return 1


In [8]:
def get_page2_data(path):
    flag=1
    NewsInfoList = []
    page2_url = page2_baseurl + path
    time.sleep(0.1)
    response = requests.get(page2_url, headers=headers)
    response.encoding = 'utf-8'
    html = response.text

    soup = BeautifulSoup(html, 'html.parser')
    block=soup.find('tbody', class_='listbody')
    datas=block.find_all('tr') if block else []

    for data in datas:
        tds=data.find_all('td')
        NewsInfoList.append(NewsInfo(tds[0].text, tds[1].text, tds[2].text, tds[3].text, tds[4].text))
        link=tds[2].find('a')['href']
        datedate=get_date(link)
        if check_data(datedate) == 1:
            NewsInfoList.append(NewsInfo(tds[0].text, tds[1].text, tds[2].text, tds[3].text, datedate))
        elif check_data(datedate) == 0:
            flag=0
            break
        elif check_data(datedate) == 2:
            continue
    return NewsInfoList,flag

In [9]:
newsMap = {}
for tab in tab_list:
    paths=get_page1_data(tab)
    for path in paths:
        id=int(re.findall(r'\d+', path)[0])
        newsMap[id]=[]
        flag=1
        for i in range(1,1000):
            if flag==0:
                break
            pathstr=path.replace('.html',',f_'+str(i)+'.html')
            news,flag=get_page2_data(pathstr)
            newsMap[id]=newsMap[id]+news

ProxyError: HTTPSConnectionPool(host='guba.eastmoney.com', port=443): Max retries exceeded with url: /list,600033,f_192.html (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001AD433A34C0>: Failed to establish a new connection: [WinError 10061] 由于目标计算机积极拒绝，无法连接。')))

In [11]:
len(newsMap)

31

In [18]:
import pandas as pd



df = pd.DataFrame.from_dict(newsMap, orient='index').reset_index()

# 修改列名
df.columns = ['id'] + ['News' + str(i) for i in range(1, len(df.columns))]

print(df)


        id                                             News1  \
0   600000  <__main__.NewsInfo object at 0x000001AD689FA0E0>   
1   600001  <__main__.NewsInfo object at 0x000001AD6BE968C0>   
2   600002  <__main__.NewsInfo object at 0x000001AD6E5604C0>   
3   600003  <__main__.NewsInfo object at 0x000001AD710DA4A0>   
4   600004  <__main__.NewsInfo object at 0x000001AD74024AC0>   
5   600005  <__main__.NewsInfo object at 0x000001AD758A97E0>   
6   600006  <__main__.NewsInfo object at 0x000001AD793E49D0>   
7   600007  <__main__.NewsInfo object at 0x000001AD7E3D68F0>   
8   600008  <__main__.NewsInfo object at 0x000001AD00B9C8E0>   
9   600009  <__main__.NewsInfo object at 0x000001AD0435AC80>   
10  600010  <__main__.NewsInfo object at 0x000001AD06647C10>   
11  600011  <__main__.NewsInfo object at 0x000001AD093836D0>   
12  600012  <__main__.NewsInfo object at 0x000001AD0B428850>   
13  600015  <__main__.NewsInfo object at 0x000001AD1046FCD0>   
14  600016  <__main__.NewsInfo object at

In [ ]:
df_news = pd.DataFrame(columns=['id', 'views', 'comments', 'title', 'author', 'time'])

for index, row in df.iterrows():
    # 直接从News对象访问属性
    news = row['News1']
    if news:  # 确保news对象存在
        new_row = pd.DataFrame({
            'id': [row['id']],
            'views': [news.views],
            'comments': [news.comments],
            'title': [news.title],
            'author': [news.author],
            'time': [news.time]
        })
        df_news = pd.concat([df_news, new_row], ignore_index=True)

print(df_news)


In [28]:
df_news = pd.DataFrame(columns=['id', 'views', 'comments', 'title', 'author', 'time'])

# 遍历原始 DataFrame
for index, row in df.iterrows():
    for i in range(1, 100):
        news_key = 'News' + str(i)
        if pd.notna(row[news_key]):
           news = row[news_key]
           new_row = pd.DataFrame({
                'id': [row['id']],
                'views': [news.views],
                'comments': [news.comments],
                'title': [news.title],
                'author': [news.author],
                'time': [news.time]
            })
        df_news = pd.concat([df_news, new_row], ignore_index=True)


          id views comments                                     title  \
0     600000   164        3                        又开启了等公告的一周，今晚会有公告么   
1     600000  1428        7  昨天大手笔买方正证券的机构和投资者绝不是听消息购买的，是它具备投资价值和做大做强   
2     600000   956       13  无论是蚂蚁收购瑞信证券还是城堡证券收购瑞信证券，对于持有瑞信证券49%股份的方正   
3     600000   843        7  老夫炒股这些年对于是洗盘、吸筹还是主力脱逃有独到见解。那就是下一步涨了就是洗盘，   
4     600000    20        0                           打板的几十万手都没走，怕个毛线   
...      ...   ...      ...                                       ...   
3064  600033    72        1                                   不能碰的券商！   
3065  600033    50        0                       明天低开水下可以博下说不定还有大肉肉吃   
3066  600033    18        0                     难道庄会因为误导的小作文拉升？骗子本人吧。   
3067  600033  1362       14  今天券商板块涨幅铁定在板块涨幅前三位！中信证券领涨5%以上，板块内小票将掀起涨停   
3068  600033    18        0                                      明天反包   

          author         time  
0       鸡叫鸡叫半夜鸡叫  03-04 04:58  
1       股友fTMFGJ  03-04 10:22  
2         爱炒股的田鼠  03-04 04:

In [29]:
import pandas as pd
from sqlalchemy import create_engine
db_user = 'user'
db_password = 'password'
db_host = 'localhost' 
db_port = '23306'
db_name = 'database'

# 创建数据库引擎
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

In [30]:
tablename = 'news'
df_news.to_sql(tablename, con=engine, if_exists='replace', index=False)

3069

In [31]:
# SQL 查询，聚合每个股票代码每天的新闻数
sql_query = """
SELECT
    id,
    DATE(time) AS news_date,
    COUNT(*) AS news_count
FROM
    news
GROUP BY
    id, DATE(time)
ORDER BY
    id, news_date;
"""


df_aa = pd.read_sql_query(sql_query, con=engine)


In [33]:
# SQL 查询，聚合每个股票代码每天的新闻数
sql_query = """
SELECT
    n.id,
    n.title,
    dd.Dretwd,
    dd.Dretnd
FROM
    news n
JOIN
    daily_data dd ON n.id = dd.Stkcd
Where
    n.id=600022
"""


df_daily = pd.read_sql_query(sql_query, con=engine)

In [34]:
df_daily

,id,title,Dretwd,Dretnd
0,600022,明天反包,0.005769,0.005769
1,600022,今天券商板块涨幅铁定在板块涨幅前三位！中信证券领涨5%以上，板块内小票将掀起涨停,0.005769,0.005769
2,600022,难道庄会因为误导的小作文拉升？骗子本人吧。,0.005769,0.005769
3,600022,明天低开水下可以博下说不定还有大肉肉吃,0.005769,0.005769
4,600022,不能碰的券商！,0.005769,0.005769
...,...,...,...,...
23953,600022,打板的几十万手都没走，怕个毛线,0.020561,0.020561
23954,600022,老夫炒股这些年对于是洗盘、吸筹还是主力脱逃有独到见解。那就是下一步涨了就是洗盘，,0.020561,0.020561
23955,600022,无论是蚂蚁收购瑞信证券还是城堡证券收购瑞信证券，对于持有瑞信证券49%股份的方正,0.020561,0.020561
23956,600022,昨天大手笔买方正证券的机构和投资者绝不是听消息购买的，是它具备投资价值和做大做强,0.020561,0.020561
